# Image Morphing
### Load modules and set parameters
Number of epochs should be much higher to give better results, but it is lowered due to long morphing time.

In [39]:
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import cv2
import argparse
import sys
from tqdm import tqdm

ORIG_WIDTH = 0
ORIG_HEIGHT = 0
TRAIN_EPOCHS = 50

im_sz = 1024
mp_sz = 96

warp_scale = 0.05
mult_scale = 0.4
add_scale = 0.4
add_first = False

### Define functions

In [ ]:
@tf.function
def warp(origins, targets, preds_org, preds_trg):
    if add_first:
        res_targets = tfa.image.dense_image_warp((origins + preds_org[:,:,:,3:6] * 2 * add_scale) * tf.maximum(0.1, 1 + preds_org[:,:,:,0:3] * mult_scale) , preds_org[:,:,:,6:8] * im_sz * warp_scale )
        res_origins = tfa.image.dense_image_warp((targets + preds_trg[:,:,:,3:6] * 2 * add_scale) * tf.maximum(0.1, 1 + preds_trg[:,:,:,0:3] * mult_scale) , preds_trg[:,:,:,6:8] * im_sz * warp_scale )
    else:
        res_targets = tfa.image.dense_image_warp(origins * tf.maximum(0.1, 1 + preds_org[:,:,:,0:3] * mult_scale) + preds_org[:,:,:,3:6] * 2 * add_scale, preds_org[:,:,:,6:8] * im_sz * warp_scale )
        res_origins = tfa.image.dense_image_warp(targets * tf.maximum(0.1, 1 + preds_trg[:,:,:,0:3] * mult_scale) + preds_trg[:,:,:,3:6] * 2 * add_scale, preds_trg[:,:,:,6:8] * im_sz * warp_scale )

    return res_targets, res_origins

def create_grid(scale):
    grid = np.mgrid[0:scale,0:scale] / (scale - 1) * 2 -1
    grid = np.swapaxes(grid, 0, 2)
    grid = np.expand_dims(grid, axis=0)
    return grid

def produce_warp_maps(origins, targets):
    class MyModel(tf.keras.Model):
        def __init__(self):
            super(MyModel, self).__init__()
            self.conv1 = tf.keras.layers.Conv2D(64, (5, 5))
            self.act1 = tf.keras.layers.LeakyReLU(alpha=0.2)
            self.conv2 = tf.keras.layers.Conv2D(64, (5, 5))
            self.act2 = tf.keras.layers.LeakyReLU(alpha=0.2)
            self.convo = tf.keras.layers.Conv2D((3 + 3 + 2) * 2, (5, 5))

        def call(self, maps):
            x = tf.image.resize(maps, [mp_sz, mp_sz])
            x = self.conv1(x)
            x = self.act1(x)
            x = self.conv2(x)
            x = self.act2(x)
            x = self.convo(x)
            return x


    model = MyModel()

    loss_object = tf.keras.losses.MeanSquaredError()
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)

    train_loss = tf.keras.metrics.Mean(name='train_loss')

    @tf.function
    def train_step(maps, origins, targets):
      with tf.GradientTape() as tape:
        preds = model(maps)
        preds = tf.image.resize(preds, [im_sz, im_sz])

        #a = tf.random.uniform([maps.shape[0]])
        #res_targets, res_origins = warp(origins, targets, preds[...,:8] * a, preds[...,8:] * (1 - a))
        res_targets_, res_origins_ = warp(origins, targets, preds[...,:8], preds[...,8:])

        # warp maps consistency checker
        res_map = tfa.image.dense_image_warp(maps, preds[:,:,:,6:8] * im_sz * warp_scale )
        res_map = tfa.image.dense_image_warp(res_map, preds[:,:,:,14:16] * im_sz * warp_scale )

        loss = loss_object(maps, res_map) * 1 + loss_object(res_targets_, targets) * 0.3 + loss_object(res_origins_, origins) * 0.3

      gradients = tape.gradient(loss, model.trainable_variables)
      optimizer.apply_gradients(zip(gradients, model.trainable_variables))

      train_loss(loss)

    maps = create_grid(im_sz)
    maps = np.concatenate((maps, origins * 0.1, targets * 0.1), axis=-1).astype(np.float32)

    epoch = 0
    template = 'Epoch {}, Loss: {}'

    t = tqdm(range(TRAIN_EPOCHS), desc=template.format(epoch, train_loss.result()))

    for i in t:
        epoch = i + 1

        t.set_description(template.format(epoch, train_loss.result()))
        t.refresh()

        train_step(maps, origins, targets)

        if (epoch < 100 and epoch % 10 == 0) or\
           (epoch < 1000 and epoch % 100 == 0) or\
           (epoch % 1000 == 0):
            preds = model(maps, training=False)[:1]
            preds = tf.image.resize(preds, [im_sz, im_sz])

    return preds

def generate_frames(origins, targets, preds, steps=100):
    # apply maps
    org_strength = tf.reshape(tf.range(steps, dtype=tf.float32), [steps, 1, 1, 1]) / (steps - 1)
    trg_strength = tf.reverse(org_strength, axis = [0])

    frames = []
    for i in tqdm(range(steps)):
        preds_org = preds * org_strength[i]
        preds_trg = preds * trg_strength[i]

        res_targets, res_origins = warp(origins, targets, preds_org[...,:8], preds_trg[...,8:])
        res_targets = tf.clip_by_value(res_targets, -1, 1)
        res_origins = tf.clip_by_value(res_origins, -1, 1)

        results = res_targets * trg_strength[i] + res_origins * org_strength[i]
        res_numpy = results.numpy()

        img = ((res_numpy[0] + 1) * 127.5).astype(np.uint8)
        frames.append(img)
    
    return frames

### Load images

In [40]:
import os

#source = tf.keras.utils.get_file(os.getcwd() + "/img_1.jpg", 
#                                 "https://raw.githubusercontent.com/volotat/DiffMorph/master/images/img_1.jpg")
#target = tf.keras.utils.get_file(os.getcwd() + "/img_2.jpg", 
#                                 "https://raw.githubusercontent.com/volotat/DiffMorph/master/images/img_2.jpg")
source = tf.keras.utils.get_file(os.getcwd() + "/img_3.jpg", 
                                 "https://raw.githubusercontent.com/volotat/DiffMorph/master/images/img_3.jpg")
target = tf.keras.utils.get_file(os.getcwd() + "/img_4.jpg", 
                                 "https://raw.githubusercontent.com/volotat/DiffMorph/master/images/img_4.jpg")
#source = tf.keras.utils.get_file(os.getcwd() + "/img_5.jpg", 
#                                 "https://raw.githubusercontent.com/volotat/DiffMorph/master/images/img_5.jpg")
#target = tf.keras.utils.get_file(os.getcwd() + "/img_6.jpg", 
#                                 "https://raw.githubusercontent.com/volotat/DiffMorph/master/images/img_6.jpg")

dom_a = cv2.imread(source, cv2.IMREAD_COLOR)
dom_b = cv2.imread(target, cv2.IMREAD_COLOR)

# Checks if input and destination image are of the same dimensions.
if (dom_a.shape[1] != dom_b.shape[1] or dom_a.shape[0] != dom_b.shape[0]):
    print("Input Image is not the same dimensions as Destination Image.")
    sys.exit(1)

# Store original height and width
ORIG_WIDTH = dom_a.shape[1]
ORIG_HEIGHT = dom_a.shape[0]

dom_a = cv2.cvtColor(dom_a, cv2.COLOR_BGR2RGB)
dom_a = cv2.resize(dom_a, (im_sz, im_sz), interpolation = cv2.INTER_AREA)
dom_a = dom_a / 127.5 - 1


dom_b = cv2.cvtColor(dom_b, cv2.COLOR_BGR2RGB)
dom_b = cv2.resize(dom_b, (im_sz, im_sz), interpolation = cv2.INTER_AREA)
dom_b = dom_b / 127.5 - 1

origins = dom_a.reshape(1, im_sz, im_sz, 3).astype(np.float32)
targets = dom_b.reshape(1, im_sz, im_sz, 3).astype(np.float32)

221231/221231 [==============================] - 0s 0us/step


### Train maps and generate frames

In [41]:
print("Training...")
preds = produce_warp_maps(origins, targets)

steps = 100
print("Morphing...")
frames = generate_frames(origins, targets, preds, steps)  # generate frames between source and target images

Training...


Epoch 50, Loss: 0.11399739235639572: 100%|██████████| 50/50 [01:07<00:00,  1.35s/it]


Morphing...


100%|██████████| 100/100 [00:21<00:00,  4.74it/s]


### Display frames

In [44]:
import copy
import ipywidgets
from IPython.display import display, clear_output
from matplotlib import pyplot

slider = ipywidgets.IntSlider(min=1, max=100, step=1)
button_show = ipywidgets.Button(description="Show")


def on_click(_):
    global slider
    clear_output()
    display(slider, button_show)
    pyplot.imshow(frames[slider.value-1])


button_show.on_click(on_click)
slider.value = 50
display(slider, button_show)

IntSlider(value=50, min=1)

Button(description='Show', style=ButtonStyle())